In [1]:
from datasets import load_dataset, DatasetDict
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer
from llama_model import *
import numpy as np
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset
dataset = load_dataset("prachathai67k")

In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("pythainlp/thainer-corpus-v2-base-model")

In [4]:
# test tokenizer
test_text = "สวัสดีชาวโลก"
test_tokens = tokenizer(test_text, add_special_tokens=True)
print(tokenizer.convert_ids_to_tokens(test_tokens["input_ids"]))

['<s>', '▁สวัสดี', 'ชาวโลก', '</s>']


In [5]:
# reduce size of data
reduced_train_dataset = dataset["train"].select(range(5000)) # use 10k row
reduced_validation_dataset = dataset["validation"].select(range(1000))
reduced_dataset = DatasetDict({
    "train": reduced_train_dataset,
    "validation": reduced_validation_dataset,
})

In [6]:
def preprocess_data(data):
    max_length = 16
    # Tokenize the input text
    inputs = tokenizer(data['body_text'], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    # Create input_ids by excluding the last token
    # Assuming the last token is not needed in input_ids for the input
    input_ids = inputs.input_ids[:, 2:-2]  # Exclude the last token for input
    # Create labels by excluding the first token and appending a new token at the end
    # Assuming the first token is not needed in labels for the output
    labels = inputs.input_ids[:, 3:-1]  # Exclude the first token for labels
    
    # Update inputs dictionary
    inputs['input_ids'] = input_ids
    inputs['labels'] = labels
    return inputs

In [7]:
tokenized_dataset = reduced_dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
tokenized_dataset

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1122.91 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [8]:
print("input_ex:",tokenizer.decode(tokenized_dataset["train"][1]['input_ids']))
print("label_ex:",tokenizer.decode(tokenized_dataset["train"][1]['labels']))

input_ex: หน่วยงานความมั่นคงสั่งเบรคการนําเสนอรายงานละเมิดสิทธิมนุษยชนในรัฐกะเหรี่ยง อ้าง
label_ex: ความมั่นคงสั่งเบรคการนําเสนอรายงานละเมิดสิทธิมนุษยชนในรัฐกะเหรี่ยง อ้างรัฐบาลพม่า


In [9]:
#reduce the size of the model since limit of GPU ram
dim = 288
n_layers =  6
n_heads =  6
n_kv_heads = n_heads
multiple_of = 32
dropout = 0.1
max_seq_len = 350

In [10]:
#set ModelArgs
model_args = ModelArgs(
    dim=dim,
    n_layers=n_layers,
    n_heads=n_heads,
    n_kv_heads=n_heads,
    vocab_size=tokenizer.vocab_size ,
    multiple_of=multiple_of,
    max_seq_len=max_seq_len,
    dropout=dropout,
) 
model = Transformer(model_args)

In [11]:
#Define optimizer and  loss_fn
optimizer = Adam(model.parameters(), lr=5e-4)
loss_fn = CrossEntropyLoss()

In [12]:
# set Dataloader of the training set and validation set
batch_size = 1
train_loader = DataLoader(tokenized_dataset["train"], batch_size=batch_size, shuffle=True)
val_loader = DataLoader(tokenized_dataset["validation"], batch_size=batch_size)

In [13]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, show_prediction):
    model.train()
    total_loss = 0
    cnt_loop =0
    for batch in tqdm(data_loader):
        cnt_loop +=1
        input_ids =torch.stack( batch['input_ids']).to(device)
        labels = torch.stack(batch['labels']).to(device)
        optimizer.zero_grad()
        outputs = model(input_ids)
        temperature=0.1
        top_k=10
        logits = outputs[:, -1, :]
        logits = logits / temperature
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        if (cnt_loop % show_prediction ==0):
            print("Input:",tokenizer.decode(input_ids.view(-1)))
            print("Label:",tokenizer.decode(labels.view(-1)))
            print("Prediction:",tokenizer.decode(next_token.view(-1)))

        
        loss = loss_fn(outputs.view(-1, model.vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
            
    return total_loss / len(data_loader)

def validate_epoch(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = torch.stack(batch['input_ids']).to(device)
            labels = torch.stack(batch['labels']).to(device)
            
            outputs = model(input_ids)

            temperature=0.1
            top_k=10
            logits = outputs[:, -1, :]
            logits = logits / temperature
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        
            loss = loss_fn(outputs.view(-1, model.vocab_size), labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(data_loader)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Transformer(
  (tok_embeddings): Embedding(25005, 288)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=288, out_features=288, bias=False)
        (wk): Linear(in_features=288, out_features=288, bias=False)
        (wv): Linear(in_features=288, out_features=288, bias=False)
        (wo): Linear(in_features=288, out_features=288, bias=False)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=288, out_features=768, bias=False)
        (w2): Linear(in_features=768, out_features=288, bias=False)
        (w3): Linear(in_features=288, out_features=768, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=288, ou

In [15]:
epochs = 70
show_prediction = 1000
best_val_loss = np.inf
best_train_loss = np.inf
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, show_prediction)
    val_loss = validate_epoch(model, val_loader, loss_fn, device)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        torch.save(model.state_dict(), "best_train_loss_model.pth")
        print("Saved Best Train loss Model")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_val_loss_model.pth")
        print("Saved Best Val loss Model")

print("Training Complete")

 20%|██████████████████████████████████▏                                                                                                                                       | 1005/5000 [00:38<02:11, 30.33it/s]

Input: ภาคใต้<unk>18 ก.ค. 2549 สมช
Label: <unk>18 ก.ค. 2549 สมช.
Prediction:   <unk>.. <unk> <unk> 


 40%|████████████████████████████████████████████████████████████████████▏                                                                                                     | 2005/5000 [01:11<01:35, 31.37it/s]

Input: ถ้อยคําถาม เมื่อชุมชนจะมีการพัฒนาต่อไปด้วยการติดตั้งไฟฟ้า ย่อม
Label: ถาม เมื่อชุมชนจะมีการพัฒนาต่อไปด้วยการติดตั้งไฟฟ้า ย่อมมีสิทธิ์
Prediction:          มิ 


 60%|██████████████████████████████████████████████████████████████████████████████████████████████████████                                                                    | 3003/5000 [01:49<01:30, 22.16it/s]

...

Input: ก่อนหน้านี้ประธานาธิบดีศรีลังกาสั่งยุบสภาและถอดถอนนายกรัฐมนตรีร
Label: นี้ประธานาธิบดีศรีลังกาสั่งยุบสภาและถอดถอนนายกรัฐมนตรีรา
Prediction: นี้ว่า ดีฟ "ไม่แล้วคดีจากกรัฐมนตรี.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [03:51<00:00, 21.62it/s]


Input: การเมือง เตรียมโละทิ้ง 251 กก. ครม
Label: เตรียมโละทิ้ง 251 กก. ครม.
Prediction: จัดละทิ้ง พ  ก. พ.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:20<00:00, 48.83it/s]


Epoch 70, Train Loss: 3.6036, Validation Loss: 8.0575
Saved Best Train loss Model
Training Complete
